In [8]:
import json
import pandas as pd


## TODO
* Make `add_rna_seq_data` and `create_or_update` more efficient
* Add arabidopsis orthologues to DB
  * Get from accession numbers (NIH and )
* Connect front end to db for RNA-seq
  * search by:
    * [X] At gene
    * [X] Xe gene
    *  GO terms
  * Select experiment
* add species.id to experiments model so that i can filter by exp for specific species


* Maybe clear the textbox on expression_page when user changes something in sidebar and add text in box physically rather than placeholder
* display genes for different GO terms in different panels?
  
## Done
* Show which queried Xe genes arent in DB
* plot expression values for Xerophyta genes
  * log or normalised
* Query expression data by experiment in DB (currently searches all of expression table)
* Fix button stopping showing raw data
* add DEGs table and DEG data
  * filter expression data by DEGs
* Fix up `gene_query_page.py`
  * move the functions to the database
  
## Questions
- How should i handle the clusters because they aren't consistent?
- I need the humilis fasta file from nicci
- Cant add schlecteri seedlings RNA seek data because idk what half the columns are
- One of mamosas files (Xs) has gene counts all of 0
- what are mamosas files
- Michaels adult leaf tissues dont have any info in the columns
- Does it make sense that X humilis has 99000 genes and other two have 28000
- The At mapping has a lot of unplaced scaffold         unknown protein  
  - any idea what these are? should they be treated to no blast hit (assumed yes)?
- X elegans has 27920 genes in annotaition file but 28572 genes in FASTA, why?
  - 'xele.ptg000010l.6', 'xele.ptg000010l.3'
  - one has GO terms, the other doesnt. Because the one is in nuclear annotation file which has GO terms and other is in arab_topblast hit which has arab annotations but no go terms

## 01-01-2025 Add RNAseq data to DB
Need to add the X. elegans RNA-seq data to the db. 

Steps:
1. Add tables to `models.py`  and add info to DB schema with Alembic  
2. Format data into long data format  

### 1. Add table
**Set up alembic** 

`alembic init alembic`  
... chatGPT for other steps  

**Create a migration script**   

`alembic revision --autogenerate -m "Your migration description here"`  
fill in script
`alembic upgrade head`  


Added two tables: `Gene_expressions` and `Experiments`

### 2. Format the data

Use the `data_tidier.py` script and the below python cells



In [ ]:
import data_tidier as dt
import pandas as pd

# Load the RNA expression file
file_path = "all_data/Michael_RNAseq/Xs seedlings/Xs_seedlings_DESeq2 normalised counts table.csv"
df = pd.read_csv(file_path)

# Transform the data to long formathead 
long_df = dt.transform_to_long(df)

long_df.to_csv(f"{file_path}_tidy", index=False)
print(long_df.head())




In [ ]:
df = dt.add_log2(long_df) # add log2 transformation
df = dt.format_time_points(df) # format timepoint
df.rename(columns={"expression": "normalised_expression"}, inplace=True)
print(df.head())

df.to_csv("all_data/Michael_RNAseq/Xe_seedlings (updated)/Xe_seedlings_DESeq2_normalised_counts_table_tidy_for_db.csv", index=False)

### 3. Add to database
Added X. elegans seedling time course data.  



In [ ]:
import db_manager as dbm


species_id = "X. elegans"
experiment_name = "xe_seedlings_time_course"
rna_seq_data = pd.read_csv("all_data/Michael_RNAseq/Xe_seedlings (updated)/Xe_seedlings_DESeq2_normalised_counts_table_tidy_for_db.csv")

dbm.add_experiment(experiment_name, "time course of X. elegans seedlings")
dbm.add_rna_seq_data(rna_seq_data, species_id, experiment_name )


## 02.01.2025 Connect frontend to DB

In [ ]:
import db_manager as dbm
import db

species_id = "X. elegans"
experiment_name = "xe_seedlings_time_course"
database = db.DB()

database.link_experiment_to_species(experiment_name, species_id)

## 5-01-2025 Add DEG table

- added DEG tables and data for X elegans seedlings.
- can filter by DEGs now
- Can also query expression data based on GO terms

#### Testing DEG implementation
"12","Xele.ptg000001l.15","ReT48","Up-regulated","None","None"
"46","Xele.ptg000001l.50","None","None","DeT09","Down-regulated"
"137","Xele.ptg000001l.159","ReT04","Up-regulated","DeT12","Down-regulated"

Xele.ptg000001l.15
Xele.ptg000001l.50
Xele.ptg000001l.159

nice example: nuclear ubiquitin ligase complex
  - 12 genes
  - 6 degs
  - 3 up 
  - 3 down

In [ ]:
import db 

database = db.DB()

gene = "Xsch.v2.MJHO01000001.1.1119"
genes =database.get_genes_by_go_term_or_description(["jasmonic acid and ethylene-dependent systemic resistance"], "X. elegans")
gene_names = [gene.gene_name for gene in genes]

print(len(gene_names))

data = database.get_gene_expression_data(gene_names, "xe_seedlings_time_course")
print(data)

In [ ]:
import models
from sqlalchemy import func

def get_go_terms_with_fewest_genes(threshold=0):
    """
    Query GO terms linked to the fewest genes in ascending order of gene count.

    Parameters:
        session: SQLAlchemy session object.

    Returns:
        List of tuples containing GO term ID, GO term name, and gene count.
    """
    database = db.DB()
    session = database.session
    query = (
        session.query(
            models.GO.go_id,          # Select the GO ID
            models.GO.go_name,        # Select the GO Name
            func.count(models.Gene.id).label("gene_count")  # Count distinct genes
        )
        .join(models.annotations_go, models.annotations_go.c.go_id == models.GO.id)  # Join GO to annotations_go
        .join(models.Annotation, models.Annotation.id == models.annotations_go.c.annotation_id)  # Join annotations_go to annotations
        .join(models.Gene, models.Gene.id == models.Annotation.gene_id)  # Join annotations to genes
        .join(models.Species, models.Species.id == models.Gene.species_id)  # Join genes to species
        .group_by(models.GO.go_id, models.GO.go_name)  # Group by GO ID and name
        .having(func.count(models.Gene.id) > threshold)  # Filter by gene count greater than 0
        .order_by(func.count(models.Gene.id).asc())  # Order by gene count in ascending order
        .filter(models.Species.name == "X. elegans") 
    )

    return query.all()

print(get_go_terms_with_fewest_genes(13))

## 24-01-2025 Add Humilis Fasta data and neaten up interface
- add humilis species, data
- TODO make adding fasta info more efficient 
- 

In [1]:
# Add the species and gene sequences to the database

import db as db
import db_manager as dbm



database = db.DB()
species_name = "X. humilis"
fasta_file = "all_data/Xhumilis_Nov2024/Xhum_CDS_annot150424.fasta"

species = database.add_species(species_name)
species_id = species.id 
# dbm.add_gene_sequence_from_fasta(fasta_file, species_id)

In [ ]:
# add the gene annotations to the database

annotation_file = "all_data/Xhumilis_Nov2024/20241111_Xhumilis_annotation_76405_export table_Oliver.csv"
dbm.add_gene_annotations(annotation_file, species_id)


In [ ]:
import db as db
from sqlalchemy.orm import aliased
from sqlalchemy.sql import func
import models
def get_counts_by_species(species_id):
    # Count GO terms
    database = db.DB()
    session = database.session
    go_count = session.query(func.count(models.GO.id)) \
        .join(models.annotations_go) \
        .join(models.Annotation) \
        .join(models.Gene) \
        .filter(models.Gene.species_id == species_id) \
        .scalar()

    # Count InterPro terms
    interpro_count = session.query(func.count(models.InterPro.id)) \
        .join(models.annotations_interpro) \
        .join(models.Annotation) \
        .join(models.Gene) \
        .filter(models.Gene.species_id == species_id) \
        .scalar()

    # Count gene annotations
    annotation_count = session.query(func.count(models.Annotation.id)) \
        .join(models.Gene) \
        .filter(models.Gene.species_id == species_id) \
        .scalar()


    return {
        "GO_terms_count": go_count,
        "InterPro_count": interpro_count,
        "Gene_annotations_count": annotation_count
    }

# Example usage
for i in range(1, 4):
    counts = get_counts_by_species(i)
    print(counts)

293425 +312558 +1017868 = 1,623,851

## 14-02-2025 Add X elegans arabidopsis genes to database

- Below code takes in a CSV file with three columns with names ["Gene name","At Locus ID","Wiki gene description"]
- It then converts it into a dataframe, passes it to DB.py where the At locus and Wiki gene description (common name) are added to the database
- the below function has also been added to the db_manager.py

In [1]:
import pandas as pd
import database.db as db

def add_a_thaliana_gene_mapping(mapping_file):
    database = db.DB()
    
    data = pd.read_csv(mapping_file)
    
    # Ensure columns exist
    required_columns = ["Gene name","At Locus ID","Wiki gene description"]
    if not all(col in data.columns for col in required_columns):
        raise ValueError(f"CSV is missing one or more required columns: {required_columns}")

    data.sort_values("At Locus ID", inplace=True, ascending=False)

    database.add_a_thaliana_gene_mappings(data)
    # add at common name
    # add locus
    # make all lower case
    # retreive gene ID, 

mapping_file = "all_data/arab_gene_mapping/20250212_Xelegans_Arabidopsis_topblast_TAIRID_Mapping.csv"
add_a_thaliana_gene_mapping(mapping_file)

Processed 0 gene mappings.
Processed 100 gene mappings.
Processed 200 gene mappings.
Processed 300 gene mappings.
Processed 400 gene mappings.
Processed 500 gene mappings.
Processed 600 gene mappings.
Processed 700 gene mappings.
Processed 800 gene mappings.
Processed 900 gene mappings.
Processed 1000 gene mappings.
Processed 1100 gene mappings.
Processed 1200 gene mappings.
Processed 1300 gene mappings.
Processed 1400 gene mappings.
Processed 1500 gene mappings.
Processed 1600 gene mappings.
Processed 1700 gene mappings.
Processed 1800 gene mappings.
Processed 1900 gene mappings.
Processed 2000 gene mappings.
Processed 2100 gene mappings.
Processed 2200 gene mappings.
Processed 2300 gene mappings.
Processed 2400 gene mappings.
Processed 2500 gene mappings.
Processed 2600 gene mappings.
Processed 2700 gene mappings.
Processed 2800 gene mappings.
Processed 2900 gene mappings.
Processed 3000 gene mappings.
Processed 3100 gene mappings.
Processed 3200 gene mappings.
Processed 3300 gene ma

## 19-02-2025 Retrieve all annotation data for a list of genes (At homologue locus, common name, description)
 

### Retrieve annotation data from xerophyta gene

In [67]:
import database.db as db

database = db.DB()
gene_name = ["Xele.ptg000049l.138","Xele.ptg000049l.140"]
# gene_name = ["Xele.ptg000049l.140"]

genes = database.get_gene_annotation_data(gene_name, "xerophyta_gene_name")

In [ ]:
data = []
for gene in genes:
    species = gene.species.name 
    gene_name=gene.gene_name
    coding_sequence = gene.coding_sequence

    for homologue in gene.arabidopsis_homologues:
        a_thaliana_locus = homologue.a_thaliana_locus
        a_thaliana_common_name = homologue.a_thaliana_common_name

    for annotation in gene.annotations:
        description = annotation.description
        e_value = annotation.e_value
        bit_score = annotation.bit_score
        similarity = annotation.similarity
        alignment_length = annotation.alignment_length
        positives = annotation.positives
        
        go_ids = ", ".join([go.go_id for go in annotation.go_ids])
        go_names = ", ".join([go.go_name for go in annotation.go_ids])

        enzyme_codes = ", ".join([enzyme_codes.enzyme_code for enzyme_codes in annotation.enzyme_codes])
        enzyme_names = ", ".join([enzyme_codes.enzyme_name for enzyme_codes in annotation.enzyme_codes])
       
        interpro_ids = ", ".join([interpro.interpro_id for interpro in annotation.interpro_ids])
        data.append({
            "species": species,
            "gene_name": gene_name,
            "coding_sequence": coding_sequence,
            "a_thaliana_locus": a_thaliana_locus,
            "a_thaliana_common_name": a_thaliana_common_name,
            "description": description,
            "e_value": e_value,
            "bit_score": bit_score,
            "similarity": similarity,
            "alignment_length": alignment_length,
            "positives": positives,
            "go_ids": go_ids,
            "go_names": go_names,
            "enzyme_codes": enzyme_codes,
            "enzyme_names": enzyme_names,
            "interpro_ids": interpro_ids
        })

In [72]:
for d in data:
    print(d)

{'species': 'X. elegans', 'gene_name': 'Xele.ptg000049l.138', 'a_thaliana_locus': 'AT3G13850', 'a_thaliana_common_name': 'LOB domain-containing protein 22', 'description': 'RecName: Full=LOB domain-containing protein 22; AltName: Full=ASYMMETRIC LEAVES 2-like protein 30; Short=AS2-like protein 30', 'e_value': 3.75e-33, 'bit_score': None, 'similarity': None, 'alignment_length': None, 'positives': None, 'go_ids': 'P:GO:0009556, P:GO:0009786, P:GO:0009799, P:GO:0009944, P:GO:0009954, P:GO:0009965, P:GO:0010089, P:GO:0010199, P:GO:0010311, P:GO:0010628, P:GO:0045893, P:GO:0048441, P:GO:1905177, P:GO:1990110, F:GO:0000976, F:GO:0005515, C:GO:0005654', 'go_names': 'P:microsporogenesis, P:regulation of asymmetric cell division, P:specification of symmetry, P:polarity specification of adaxial/abaxial axis, P:proximal/distal pattern formation, P:leaf morphogenesis, P:xylem development, P:organ boundary specification between lateral organs and the meristem, P:lateral root formation, P:positive r

### Retreive annotation data from Arabidopsis homologue

In [15]:
import database.db as db

database = db.DB()

gene_list = ["AT1G17650", "AT3G13850", "not in"]
genes = database.get_gene_annotation_data_from_a_thaliana_homologue(gene_list)
for g in genes:
    print(f'{g.arabidopsis_homologues[0].a_thaliana_locus} {g.gene_name} ')

matched_loci = {gene.arabidopsis_homologues[0].a_thaliana_locus.lower() for gene in genes}
print(matched_loci)
missing_loci = {gene for gene in gene_list if gene.lower() not in matched_loci}
print(missing_loci)



AT3G13850 Xele.ptg000018l.1938 
AT3G13850 Xele.ptg000028l.172 
AT3G13850 Xele.ptg000049l.138 
AT1G17650 Xele.ptg000049l.140 
{'at3g13850', 'at1g17650'}
{'not in'}


In [5]:
print(len(genes))

4159


## 23-02-2025 Missing annotation

- genes without blast hits dont have an associated annotation object, causing issues when retreiving annotation data
- TODO
  - confirm which genes dont have associated annotation object
  - check which annotation file contains the gene name or not
  - 27920 genes in annotaition file but 28572 genes in FASTA/database
  - assume the missing 652 arent annotated? 
    - they wont have an At match because the AT matches only has the  27920 but some of the 27920 genes (4159 genes) also have no blast hit
  - CHECK how many Xele genes have annotation in DB i.e select annotation count where species.id = 1


Annotated = 23760
No blast hit = 4428

In [1]:
import database.db as db 

database = db.DB()
x = "Xele.ptg000041l.177" #no annotation
y ="Xele.ptg000049l.139" # no annotation
gene = database.get_gene_by_name(y)

In [11]:
print(gene.arabidopsis_homologues[0].a_thaliana_locus)
print(gene.id)


no blast hit
4641


In [ ]:
import database.models as  models

database.session.query(models.Gene).where(species).count()

## 25-02-2025 Add remaining annotation data



In [ ]:
import pandas as pd
import database.db as db
import database.models as models
from database.db_manager import map_genes_to_ids
# annotationfile = "all_data/Xelegans_Nov2024/20240918_Xelegans_Arabidopsis_topblast_nuclear_blastx_Oliver.csv"
# annotationfile = "all_data/Xhumilis_Nov2024/20241111_Xhumilis_Arabidopsis_topblast_blastx_Oliver.csv"
annotationfile = "all_data/Xschlechteri_Nov2024/20241108_Xschlechteri_Arabidopsis_topblast_blastx_Oliver.csv"
species_id = 2


df = pd.read_csv(annotationfile)
df = df[['Sequence name', 'E-Value', "Sequence desc.",
        'Similarity', 'Bit-Score', 'Alignment length',
       'Positives']]

df = df.rename(columns={"Sequence name": "gene_name", 
                        "Sequence desc.":"description",
                        "E-Value": "E value", 
                        "Similarity": "similarity", 
                        "Bit-Score": "bit_score", 
                        "Alignment length": "alignment_length", 
                        "Positives": "positives"})

records =[]


records = records[2:4]

gene_ids = map_genes_to_ids(species_id)

for _, row in df.iterrows():
        
        # map SeqName to gene_id
        print(len(gene_ids))
        gene_id = gene_ids.get(row["gene_name"])
        if not gene_id:
            print(f"Gene {row['gene_name']} not found in database")
            continue

        annotation_data = {
            "gene_id": gene_id,
            "description": row["description"],
                "e_value": row["E value"],
                "similarity": row["similarity"],
                "bit_score": row["bit_score"],
                "alignment_length": row["alignment_length"],
                "positives": row["positives"]
                }
        
        records.append(annotation_data)


database = db.DB()     
# database.create_or_update(models.Annotation, records, lookup_fields=["gene_id"])

28572
Gene Xsch.v2.MJHO01000506.1.1 not found in database
28572
Gene Xsch.v2.MJHO01000879.1.1 not found in database
28572
Gene Xsch.v2.MJHO01000743.1.1 not found in database
28572
Gene Xsch.v2.MJHO01000072.1.1 not found in database
28572
Gene Xsch.v2.MJHO01000072.1.2 not found in database
28572
Gene Xsch.v2.MJHO01000072.1.3 not found in database
28572
Gene Xsch.v2.MJHO01000072.1.4 not found in database
28572
Gene Xsch.v2.MJHO01000072.1.5 not found in database
28572
Gene Xsch.v2.MJHO01000072.1.7 not found in database
28572
Gene Xsch.v2.MJHO01000072.1.8 not found in database
28572
Gene Xsch.v2.MJHO01000072.1.9 not found in database
28572
Gene Xsch.v2.MJHO01000072.1.10 not found in database
28572
Gene Xsch.v2.MJHO01000072.1.11 not found in database
28572
Gene Xsch.v2.MJHO01000072.1.13 not found in database
28572
Gene Xsch.v2.MJHO01000072.1.14 not found in database
28572
Gene Xsch.v2.MJHO01000072.1.15 not found in database
28572
Gene Xsch.v2.MJHO01000072.1.16 not found in database
28572
Ge